In [1]:
from dotenv import load_dotenv
from pydantic import BaseModel

load_dotenv()

True

# How to implement a task using an Aleph Alpha model
0. Define the task (see [here](./how_to_define_a_task.ipynb))
1. Decide which model best suits your use case (for a list of Aleph Alpha models see [here]())
2. Define a `Task` subclass
   1. Pass the Model to the constructor 
   2. Implement your domain logic in `do_run()`
      1. Generate a `Prompt` via `ControlModel.to_instruct_prompt()`, `PromptTemplate.to_rich_prompt()` or `Prompt.from_text()`
      2. Excute the Prompt with the model
      3. Map the prompt output to the task output class
3. Run and test it

# How to log and debug a task
1. Use `task_span.log()` to create custome logs in the trace. The task input and output are automatically logged.
2. Pass the `task_span` to `subtask.run()` to ensure continous logging through out all steps. 
3. Use the trace viewer to view and debug the trace
   1. Use and display the `InMemoryTracer` in a notebook to automatically send the trace data to the trace viewer
   2. To create persistant traces use the `FileTracer` instead

# How to use subtasks in your task
1. Follow [How to implement a task using an Aleph Alpha model](#how-to-implement-a-task-using-an-aleph-alpha-model) and replace the model class with the appropriate subtask.

In [12]:
from intelligence_layer.core import Task
from intelligence_layer.core.model import CompleteInput, LuminousControlModel
from intelligence_layer.core.tracer.tracer import TaskSpan

# Define the input and output types for your task

class TellAJokeTaskInput(BaseModel):
    """This is the text we will extract keywords from"""

    topic: str


class TellAJokeTaskOutput(BaseModel):
    """The matching set of keywords we aim to extract"""

    joke: str

class TellAJoke(Task[TellAJokeTaskInput, TellAJokeTaskOutput]):
    PROMPT_TEMPLATE: str = """Tell me a joke about the following topic:"""

    def __init__(
        self, model: LuminousControlModel = LuminousControlModel()
    ) -> None:
        self._model = model

    def do_run(
        self, input: TellAJokeTaskInput, task_span: TaskSpan
    ) -> TellAJokeTaskOutput:
        prompt = self._model.to_instruct_prompt(self.PROMPT_TEMPLATE, input.topic)
        completion_input = CompleteInput(prompt = prompt)
        completion = self._model.complete(completion_input, task_span)
        return TellAJokeTaskOutput(joke=completion.completions[0].completion)

In [15]:
from intelligence_layer.core.tracer.tracer import NoOpTracer


TellAJoke().run(TellAJokeTaskInput(topic="Dolphins"), NoOpTracer())

model_version='20240215' completions=[CompletionResult(log_probs=None, completion=' Why did the dolphin go to the dentist? Because he wanted to keep his flipper!', completion_tokens=None, finish_reason='end_of_text', raw_completion=None)] num_tokens_prompt_total=20 num_tokens_generated=20 optimized_prompt=None


TellAJokeTaskOutput(joke=' Why did the dolphin go to the dentist? Because he wanted to keep his flipper!')

## Using existing tasks as subtasks in your own task 

- define a task as described [here](./how_to_define_a_task.ipynb)